In [32]:
import numpy as np 
import pandas as pd 
from scipy.stats import norm

In [2]:
N_exp = 10000
N_con = 10000

In [3]:
click_exp = pd.Series(np.random.binomial(1,0.4,size = N_exp)) # success of 0.4
click_con = pd.Series(np.random.binomial(1,0.2,size = N_con)) # success of 0.2

In [5]:
exp_id = pd.Series(np.repeat('exp',N_exp))
con_id = pd.Series(np.repeat('con',N_con))

In [8]:
df_exp = pd.concat([click_exp, exp_id], axis=1)
df_con = pd.concat([click_con, con_id], axis=1)

In [12]:
df_exp.columns = ["click", "group"]
df_con.columns = ["click", "group"]

In [13]:
df_ab_testing = pd.concat([df_exp, df_con], axis = 0).reset_index(drop=True)
df_ab_testing

,click,group
0,0,exp
1,1,exp
2,1,exp
3,1,exp
4,0,exp
...,...,...
19995,0,con
19996,1,con
19997,0,con
19998,0,con


In [14]:
x_con = df_ab_testing.groupby('group')['click'].sum().loc['con']
x_exp = df_ab_testing.groupby('group')['click'].sum().loc['exp']

In [16]:
x_con, x_exp

(1985, 4039)

In [19]:
df_ab_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   click   20000 non-null  int32 
 1   group   20000 non-null  object
dtypes: int32(1), object(1)
memory usage: 234.5+ KB


In [20]:
df_ab_testing.groupby("group")['click'].sum()

group
con    1985
exp    4039
Name: click, dtype: int32

In [23]:
p_con_hat = x_con/N_con
p_exp_hat = x_exp/N_exp

round(p_con_hat, 2), round(p_exp_hat, 2)

(0.2, 0.4)

In [25]:
p_pooled_hat = (x_con + x_exp)/(N_con + N_exp)

pooled_variance = p_pooled_hat * (1-p_pooled_hat) * (1/N_con + 1/N_exp)

f"p^_pooled is: {p_pooled_hat} and pooled_variance is: {pooled_variance}"

'p^_pooled is: 0.3012 and pooled_variance is: 4.2095712000000004e-05'

In [26]:
std = np.sqrt(pooled_variance)
std

0.006488120837345741

In [27]:
t_test = (p_con_hat - p_exp_hat)/std
t_test

-31.65785674300544

In [28]:
alpha = 0.05

In [33]:
z_crit = norm.ppf(1-alpha/1)
z_crit

1.6448536269514722

In [36]:
p_value = 2 * norm.sf(abs(t_test))
round(p_value,2)

0.0

In [37]:
CI = [round((p_exp_hat - p_con_hat) - std * z_crit, 3), round((p_exp_hat - p_con_hat)+std*z_crit,3)]

In [38]:
CI

[0.195, 0.216]

In [ ]:
delta = 0.03 # this is lower meaning it is practically significant 